In [8]:
import pandas as pd
import numpy as np

# Load the data

customer_db = pd.read_csv("data/DM_AIAI_CustomerDB.csv")
flights_db = pd.read_csv("data/DM_AIAI_FlightsDB.csv")


In [9]:
customer_db.head()

,Unnamed: 0,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,...,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
0,0,480934,Cecilia,Householder,Cecilia Householder,Canada,Ontario,Toronto,43.653225,-79.383186,...,female,Bachelor,Urban,70146.0,Married,Star,2/15/2019,NaN,3839.14,Standard
1,1,549612,Dayle,Menez,Dayle Menez,Canada,Alberta,Edmonton,53.544388,-113.490930,...,male,College,Rural,0.0,Divorced,Star,3/9/2019,NaN,3839.61,Standard
2,2,429460,Necole,Hannon,Necole Hannon,Canada,British Columbia,Vancouver,49.282730,-123.120740,...,male,College,Urban,0.0,Single,Star,7/14/2017,1/8/2021,3839.75,Standard
3,3,608370,Queen,Hagee,Queen Hagee,Canada,Ontario,Toronto,43.653225,-79.383186,...,male,College,Suburban,0.0,Single,Star,2/17/2016,NaN,3839.75,Standard
4,4,530508,Claire,Latting,Claire Latting,Canada,Quebec,Hull,45.428730,-75.713364,...,male,Bachelor,Suburban,97832.0,Married,Star,10/25/2017,NaN,3842.79,2021 Promotion


In [10]:
#Duplicates
customer_db.duplicated().sum()

np.int64(0)

In [11]:
#Types
customer_db.dtypes

Unnamed: 0                   int64
Loyalty#                     int64
First Name                  object
Last Name                   object
Customer Name               object
Country                     object
Province or State           object
City                        object
Latitude                   float64
Longitude                  float64
Postal code                 object
Gender                      object
Education                   object
Location Code               object
Income                     float64
Marital Status              object
LoyaltyStatus               object
EnrollmentDateOpening       object
CancellationDate            object
Customer Lifetime Value    float64
EnrollmentType              object
dtype: object

In [12]:
#Checking for missing values
customer_db.replace('', np.nan, inplace=True)
customer_db.isna().sum()

Unnamed: 0                     0
Loyalty#                       0
First Name                     0
Last Name                      0
Customer Name                  0
Country                        0
Province or State              0
City                           0
Latitude                       0
Longitude                      0
Postal code                    0
Gender                         0
Education                      0
Location Code                  0
Income                        20
Marital Status                 0
LoyaltyStatus                  0
EnrollmentDateOpening          0
CancellationDate           14611
Customer Lifetime Value       20
EnrollmentType                 0
dtype: int64

In [14]:
customer_db.describe(include="all")

,Unnamed: 0,Loyalty#,First Name,Last Name,Customer Name,Country,Province or State,City,Latitude,Longitude,...,Gender,Education,Location Code,Income,Marital Status,LoyaltyStatus,EnrollmentDateOpening,CancellationDate,Customer Lifetime Value,EnrollmentType
count,16921.000000,16921.000000,16921,16921,16921,16921,16921,16921,16921.000000,16921.000000,...,16921,16921,16921,16901.000000,16921,16921,16921,2310,16901.000000,16921
unique,NaN,NaN,4941,15404,16921,1,11,29,NaN,NaN,...,2,5,3,NaN,3,3,2449,1260,NaN,2
top,NaN,NaN,Deon,Salberg,Emma Martin,Canada,Ontario,Toronto,NaN,NaN,...,female,Bachelor,Suburban,NaN,Married,Star,4/3/2015,7/7/2020,NaN,Standard
freq,NaN,NaN,13,4,1,16921,5468,3390,NaN,NaN,...,8497,10586,5716,NaN,9842,7761,34,8,NaN,15773
mean,8440.023639,550197.393771,NaN,NaN,NaN,NaN,NaN,NaN,47.174500,-91.814768,...,NaN,NaN,NaN,37758.038400,NaN,NaN,NaN,NaN,7990.460188,NaN
std,4884.775439,259251.503597,NaN,NaN,NaN,NaN,NaN,NaN,3.307971,22.242429,...,NaN,NaN,NaN,30368.992499,NaN,NaN,NaN,NaN,6863.173093,NaN
min,0.000000,100011.000000,NaN,NaN,NaN,NaN,NaN,NaN,42.984924,-135.056840,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,1898.010000,NaN
25%,4210.000000,326823.000000,NaN,NaN,NaN,NaN,NaN,NaN,44.231171,-120.237660,...,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,3979.720000,NaN
50%,8440.000000,550896.000000,NaN,NaN,NaN,NaN,NaN,NaN,46.087818,-79.383186,...,NaN,NaN,NaN,34161.000000,NaN,NaN,NaN,NaN,5780.180000,NaN
75%,12670.000000,772438.000000,NaN,NaN,NaN,NaN,NaN,NaN,49.282730,-74.596184,...,NaN,NaN,NaN,62396.000000,NaN,NaN,NaN,NaN,8945.690000,NaN


In [3]:
flights_db.head()

,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
0,413052,2021,12,12/1/2021,2.0,2.0,9384.0,938.0,0.0,0.0
1,464105,2021,12,12/1/2021,0.0,0.0,0.0,0.0,0.0,0.0
2,681785,2021,12,12/1/2021,10.0,3.0,14745.0,1474.0,0.0,0.0
3,185013,2021,12,12/1/2021,16.0,4.0,26311.0,2631.0,3213.0,32.0
4,216596,2021,12,12/1/2021,9.0,0.0,19275.0,1927.0,0.0,0.0


In [15]:
# Duplicates
flights_db.duplicated().sum()

np.int64(2903)

In [16]:
#DataTypes
flights_db.dtypes

Loyalty#                      int64
Year                          int64
Month                         int64
YearMonthDate                object
NumFlights                  float64
NumFlightsWithCompanions    float64
DistanceKM                  float64
PointsAccumulated           float64
PointsRedeemed              float64
DollarCostPointsRedeemed    float64
dtype: object

In [17]:
# Missing values
flights_db.replace('', np.nan, inplace=True)
flights_db.isna().sum()

Loyalty#                    0
Year                        0
Month                       0
YearMonthDate               0
NumFlights                  0
NumFlightsWithCompanions    0
DistanceKM                  0
PointsAccumulated           0
PointsRedeemed              0
DollarCostPointsRedeemed    0
dtype: int64

In [18]:
# Strange Values and Descriptive statistics
flights_db.describe(include="all")

,Loyalty#,Year,Month,YearMonthDate,NumFlights,NumFlightsWithCompanions,DistanceKM,PointsAccumulated,PointsRedeemed,DollarCostPointsRedeemed
count,608436.000000,608436.000000,608436.000000,608436,608436.000000,608436.000000,608436.000000,608436.000000,608436.000000,608436.000000
unique,NaN,NaN,NaN,36,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,NaN,12/1/2021,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,NaN,16901,NaN,NaN,NaN,NaN,NaN,NaN
mean,550037.873084,2020.000000,6.500000,NaN,3.908107,0.983944,7939.341419,793.777781,235.251678,2.324835
std,258935.180575,0.816497,3.452055,NaN,5.057889,2.003785,10260.421873,1025.918521,983.233374,9.725168
min,100018.000000,2019.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,326961.000000,2019.000000,3.750000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,550834.000000,2020.000000,6.500000,NaN,0.000000,0.000000,856.400000,85.275000,0.000000,0.000000
75%,772194.000000,2021.000000,9.250000,NaN,7.200000,0.900000,15338.175000,1533.712500,0.000000,0.000000
